# Word Count MapReduce Program in Pyspark

In [ ]:
Objective:
Use Pyspark to read an input file and count frequency of unique keywords.

In [1]:
spark

### Create spark context

In [2]:
sc = spark.sparkContext

### Read the input file

In [3]:
lines = sc.textFile(filePath + "\Text1.txt")
lines.collect()

[u'The Project Gutenberg EBook of Pride and Prejudice, by Jane Austen',
 u'',
 u'This eBook is for the use of anyone anywhere at no cost and with',
 u'almost no restrictions whatsoever.  You may copy it, give it away or',
 u're-use it under the terms of the Project Gutenberg License included',
 u'with this eBook or online at www.gutenberg.org',
 u'',
 u'',
 u'Title: Pride and Prejudice',
 u'',
 u'Author: Jane Austen',
 u'',
 u'Posting Date: August 26, 2008 [EBook #1342]',
 u'Release Date: June, 1998',
 u'Last updated: February 15, 2015]',
 u'',
 u'Language: English',
 u'',
 u'Character set encoding: ASCII',
 u'',
 u'*** START OF THIS PROJECT GUTENBERG EBOOK PRIDE AND PREJUDICE ***',
 u'',
 u'',
 u'',
 u'',
 u'Produced by Anonymous Volunteers',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'PRIDE AND PREJUDICE',
 u'',
 u'By Jane Austen',
 u'',
 u'',
 u'',
 u'Chapter 1',
 u'',
 u'',
 u'It is a truth universally acknowledged, that a single man in possession',
 u'of a good fortune, must be in want of a 

### Get all the words from one line

In [4]:
words= lines.flatMap(lambda x:x.split(" "))
words.collect()

[u'The',
 u'Project',
 u'Gutenberg',
 u'EBook',
 u'of',
 u'Pride',
 u'and',
 u'Prejudice,',
 u'by',
 u'Jane',
 u'Austen',
 u'',
 u'This',
 u'eBook',
 u'is',
 u'for',
 u'the',
 u'use',
 u'of',
 u'anyone',
 u'anywhere',
 u'at',
 u'no',
 u'cost',
 u'and',
 u'with',
 u'almost',
 u'no',
 u'restrictions',
 u'whatsoever.',
 u'',
 u'You',
 u'may',
 u'copy',
 u'it,',
 u'give',
 u'it',
 u'away',
 u'or',
 u're-use',
 u'it',
 u'under',
 u'the',
 u'terms',
 u'of',
 u'the',
 u'Project',
 u'Gutenberg',
 u'License',
 u'included',
 u'with',
 u'this',
 u'eBook',
 u'or',
 u'online',
 u'at',
 u'www.gutenberg.org',
 u'',
 u'',
 u'Title:',
 u'Pride',
 u'and',
 u'Prejudice',
 u'',
 u'Author:',
 u'Jane',
 u'Austen',
 u'',
 u'Posting',
 u'Date:',
 u'August',
 u'26,',
 u'2008',
 u'[EBook',
 u'#1342]',
 u'Release',
 u'Date:',
 u'June,',
 u'1998',
 u'Last',
 u'updated:',
 u'February',
 u'15,',
 u'2015]',
 u'',
 u'Language:',
 u'English',
 u'',
 u'Character',
 u'set',
 u'encoding:',
 u'ASCII',
 u'',
 u'***',
 u'ST

###  Define function cleandata for following:

1. Convert all words to lowercase

2. Remove all double quotes 

3. Special characters{",", ".", ";", "?"} to be ignored and removed 
   from the END of the words such as 

In [5]:
def cleandata(x):
    x= x.strip().lower().replace('"','')
    char_list = [",", ".", ";", "?"]
    if  len(x) > 0 and x[-1] in char_list:
        x = x[:-1]
    return x


### Apply function cleandata on words rdd and return clean_words rdd after cleaning

In [6]:
clean_words = words.map(lambda x: cleandata(x))
clean_words.collect()

[u'the',
 u'project',
 u'gutenberg',
 u'ebook',
 u'of',
 u'pride',
 u'and',
 u'prejudice',
 u'by',
 u'jane',
 u'austen',
 u'',
 u'this',
 u'ebook',
 u'is',
 u'for',
 u'the',
 u'use',
 u'of',
 u'anyone',
 u'anywhere',
 u'at',
 u'no',
 u'cost',
 u'and',
 u'with',
 u'almost',
 u'no',
 u'restrictions',
 u'whatsoever',
 u'',
 u'you',
 u'may',
 u'copy',
 u'it',
 u'give',
 u'it',
 u'away',
 u'or',
 u're-use',
 u'it',
 u'under',
 u'the',
 u'terms',
 u'of',
 u'the',
 u'project',
 u'gutenberg',
 u'license',
 u'included',
 u'with',
 u'this',
 u'ebook',
 u'or',
 u'online',
 u'at',
 u'www.gutenberg.org',
 u'',
 u'',
 u'title:',
 u'pride',
 u'and',
 u'prejudice',
 u'',
 u'author:',
 u'jane',
 u'austen',
 u'',
 u'posting',
 u'date:',
 u'august',
 u'26',
 u'2008',
 u'[ebook',
 u'#1342]',
 u'release',
 u'date:',
 u'june',
 u'1998',
 u'last',
 u'updated:',
 u'february',
 u'15',
 u'2015]',
 u'',
 u'language:',
 u'english',
 u'',
 u'character',
 u'set',
 u'encoding:',
 u'ascii',
 u'',
 u'***',
 u'start',


### Create final_words rdd to ignore any word of length less than 3 from clean_words rdd

In [7]:
final_words = clean_words.filter(lambda x: len(x)>=3)
final_words.collect()

[u'the',
 u'project',
 u'gutenberg',
 u'ebook',
 u'pride',
 u'and',
 u'prejudice',
 u'jane',
 u'austen',
 u'this',
 u'ebook',
 u'for',
 u'the',
 u'use',
 u'anyone',
 u'anywhere',
 u'cost',
 u'and',
 u'with',
 u'almost',
 u'restrictions',
 u'whatsoever',
 u'you',
 u'may',
 u'copy',
 u'give',
 u'away',
 u're-use',
 u'under',
 u'the',
 u'terms',
 u'the',
 u'project',
 u'gutenberg',
 u'license',
 u'included',
 u'with',
 u'this',
 u'ebook',
 u'online',
 u'www.gutenberg.org',
 u'title:',
 u'pride',
 u'and',
 u'prejudice',
 u'author:',
 u'jane',
 u'austen',
 u'posting',
 u'date:',
 u'august',
 u'2008',
 u'[ebook',
 u'#1342]',
 u'release',
 u'date:',
 u'june',
 u'1998',
 u'last',
 u'updated:',
 u'february',
 u'2015]',
 u'language:',
 u'english',
 u'character',
 u'set',
 u'encoding:',
 u'ascii',
 u'***',
 u'start',
 u'this',
 u'project',
 u'gutenberg',
 u'ebook',
 u'pride',
 u'and',
 u'prejudice',
 u'***',
 u'produced',
 u'anonymous',
 u'volunteers',
 u'pride',
 u'and',
 u'prejudice',
 u'jane',

### Generate key value pairs with each word as key and 1 as value

In [8]:
pairs =final_words.map(lambda x: (x,1))
pairs.collect()

[(u'the', 1),
 (u'project', 1),
 (u'gutenberg', 1),
 (u'ebook', 1),
 (u'pride', 1),
 (u'and', 1),
 (u'prejudice', 1),
 (u'jane', 1),
 (u'austen', 1),
 (u'this', 1),
 (u'ebook', 1),
 (u'for', 1),
 (u'the', 1),
 (u'use', 1),
 (u'anyone', 1),
 (u'anywhere', 1),
 (u'cost', 1),
 (u'and', 1),
 (u'with', 1),
 (u'almost', 1),
 (u'restrictions', 1),
 (u'whatsoever', 1),
 (u'you', 1),
 (u'may', 1),
 (u'copy', 1),
 (u'give', 1),
 (u'away', 1),
 (u're-use', 1),
 (u'under', 1),
 (u'the', 1),
 (u'terms', 1),
 (u'the', 1),
 (u'project', 1),
 (u'gutenberg', 1),
 (u'license', 1),
 (u'included', 1),
 (u'with', 1),
 (u'this', 1),
 (u'ebook', 1),
 (u'online', 1),
 (u'www.gutenberg.org', 1),
 (u'title:', 1),
 (u'pride', 1),
 (u'and', 1),
 (u'prejudice', 1),
 (u'author:', 1),
 (u'jane', 1),
 (u'austen', 1),
 (u'posting', 1),
 (u'date:', 1),
 (u'august', 1),
 (u'2008', 1),
 (u'[ebook', 1),
 (u'#1342]', 1),
 (u'release', 1),
 (u'date:', 1),
 (u'june', 1),
 (u'1998', 1),
 (u'last', 1),
 (u'updated:', 1),
 (u'f

### Create freq rdd from pairs rdd with reducebykey method to get key(word), value(frequency) pair

In [9]:
freq =pairs.reduceByKey(lambda x, y:x+y)
freq.collect()

[(u'looked,--she', 1),
 (u'pardon', 16),
 (u'expostulation', 1),
 (u"jane's", 28),
 (u'bathing-place', 1),
 (u'four', 33),
 (u'brightening', 1),
 (u'straws', 1),
 (u'sleep', 4),
 (u'yes!', 2),
 (u"friend's", 8),
 (u'mansion', 2),
 (u'appetite', 2),
 (u'hate', 8),
 (u'transports', 2),
 (u'looking', 29),
 (u'unfeeling', 3),
 (u'reproofs', 2),
 (u'sweetest', 2),
 (u'seriously', 13),
 (u'presents', 3),
 (u'neighbours', 10),
 (u'whatsoever', 2),
 (u'sorry', 34),
 (u'sinking', 3),
 (u'conjecture', 7),
 (u'_me_', 15),
 (u'rational', 10),
 (u'compassion', 14),
 (u'desirable', 11),
 (u'void', 2),
 (u'proprietor--when', 1),
 (u'preservation', 2),
 (u'suggesting', 1),
 (u'probability', 9),
 (u'every', 198),
 (u'foul', 1),
 (u'softened', 6),
 (u'frailty', 1),
 (u'conception', 1),
 (u'bringing', 12),
 (u'vast', 2),
 (u'tickets', 3),
 (u"daughter's", 7),
 (u'school', 2),
 (u'tiresome', 1),
 (u'cause', 21),
 (u'conceive', 3),
 (u'protest', 1),
 (u'solid', 1),
 (u'persisted', 2),
 (u'red', 3),
 (u'gui

### Create a list of 10 given words& their frequency using lookup

In [10]:
wordlist =["advantage", "book","mistake", "dancing", "gutenberg", "astonishment" ,"hill", "yesterday", "the", "fox"]
lst = []
for i in wordlist:
    lst.append((i, freq.lookup(i)))      
lst

[('advantage', [34]),
 ('book', [15]),
 ('mistake', [7]),
 ('dancing', [21]),
 ('gutenberg', [28]),
 ('astonishment', [30]),
 ('hill', [9]),
 ('yesterday', [13]),
 ('the', [4495]),
 ('fox', [])]

In [11]:
### Create word_filtered rdd to put back list of words & their frequency to a rdd

In [12]:
word_filtered_rdd = sc.parallelize(lst,1)
word_filtered_rdd.collect()

[('advantage', [34]),
 ('book', [15]),
 ('mistake', [7]),
 ('dancing', [21]),
 ('gutenberg', [28]),
 ('astonishment', [30]),
 ('hill', [9]),
 ('yesterday', [13]),
 ('the', [4495]),
 ('fox', [])]

### Define a function println() to print both the word & its fequency in the required format

In [13]:
def println(x):
    if len(x[1]) == 0:
        final_str = str(x[0]) + " " + "0"
    else:
        final_str = str(x[0]) + " " + "".join(str(x[1][0]))
    return final_str

### Print the words

In [14]:
final_rdd = word_filtered_rdd.map(lambda x: println(x))
final_rdd.collect()

['advantage 34',
 'book 15',
 'mistake 7',
 'dancing 21',
 'gutenberg 28',
 'astonishment 30',
 'hill 9',
 'yesterday 13',
 'the 4495',
 'fox 0']

## Save the output

In [ ]:
final_rdd.saveAsTextFile("File path\Name")